In [1]:
import sys
%pylab inline
import os
import pandas as pd

import multiprocessing
import re
import json
from statsmodels.distributions.empirical_distribution import ECDF
import seaborn as sns
import locale
from locale import atof


Populating the interactive namespace from numpy and matplotlib


In [2]:
INFINITY = 1e10 # No station value should realistically be higher than this

In [3]:
def check_unique(frame):
    if frame['facility_id'].nunique() != len(frame):
        print(frame['facility_id'].value_counts())
        raise ValueError("Duplicate Fac ID!")
    if (frame['upper_bound_value'] == frame['lower_bound_value']).any():
        display(frame[frame['upper_bound_value'] == frame['lower_bound_value']])
        raise ValueError("Bounds aren't different!")
    if (frame['upper_bound_value'] < frame['lower_bound_value']).any():
        display(frame[frame['upper_bound_value'] < frame['lower_bound_value']])
        raise ValueError("Bounds are wrong!")
    if (frame['upper_bound_value'] < 0).any() or (frame['lower_bound_value'] < 0).any():
        raise ValueError("Negative bounds! are wrong!")

In [4]:
COLUMNS = ['facility_id', 'lower_bound_value', 'upper_bound_value']

In [7]:
## Data parsing
stations_df = pd.read_csv('../simulator/src/dist/simulator_data/actual_data/reverse-stations.csv')

df = pd.read_csv('../simulator/src/dist/simulator_data/actual_data/reverse-bids.csv')
df = df.drop(['auction_id', 'random_number', 'frn'], axis=1)

# 77 + 59 stations are LVHF HVHF from the very beginning, so we cannot infer anything about their home band values from bids. 49 of these stations refused off-air entirely, so we can infer nothing. 

# Restrict to rounds that at least begin with that station holding off-air 
df = df[df['round_held_option'] == 'Go off-air']

# Add in a fake round 0 with opening prices
df = pd.concat((df.copy(), pd.DataFrame({'stage': [1] * len(stations_df), 'round': [0] * len(stations_df), 'facility_id': stations_df['facility_id'], 'pre_auction_band': stations_df['pre_auction_band'], 'bid_option_price': stations_df['off_air_opening_price']})), sort=True)
# Useful for finding the pre-drop-out-round since you don't have to worry about stage/round being a double index
df['total_round_order'] = df[['stage', 'round']].apply(tuple, axis=1).rank(method='dense',ascending=True).astype(int)
df = df.merge(stations_df[['facility_id','volume']])

# Get rid of all of the VHF stations
df = df[df['pre_auction_band'] == 'UHF']

winners_df = pd.read_csv('../simulator/src/dist/simulator_data/actual_data/reverse-winning_bids.csv')

summary_df = pd.read_csv('../simulator/src/dist/simulator_data/actual_data/reverse-auction_summary.csv')
summary_df = summary_df[['stage', 'round', 'base_clock_price']]
summary_df['decrement'] = summary_df['base_clock_price'].diff() 
summary_df.loc[(summary_df['round'] == 1) & (summary_df['stage'] == 1), 'decrement'] = -45
# Add a fake stage 1 round 0 with open
summary_df = pd.concat((pd.DataFrame({'stage': [1], 'round': [0], 'base_clock_price': [900], 'decrement': [0]}),summary_df))
summary_df['total_round_order'] = summary_df[['stage', 'round']].apply(tuple, axis=1).rank(method='dense',ascending=True).astype(int)

df = df.merge(summary_df[['total_round_order', 'base_clock_price']], left_on='total_round_order', right_on='total_round_order')

ZERO_BASE_CLOCK_ROUNDS = summary_df.loc[summary_df['base_clock_price'] == 0][['stage','round']].to_records(index=False).tolist()

In [8]:
prices_df = pd.read_csv('/Users/newmanne/research/interference-data/opening_prices.csv')
prices_df = prices_df.rename(columns={'Facility ID': 'facility_id', 'Interference -Free Population': 'population', 'Go Off-Air': 'p_open', 'DMA1':'dma'})
prices_df = prices_df[prices_df['p_open'] != 'Not Needed']
locale.setlocale(locale.LC_NUMERIC, '')
prices_df['p_open'] = prices_df['p_open'].str.strip('$').str.strip().apply(atof)
prices_df['population'] = pd.to_numeric(prices_df['population'].apply(atof))
prices_df['pre_auction_band'] = prices_df['Channel'].apply(lambda x: 'UHF' if x >= 14 else 'HVHF' if x >= 7 else 'LVHF')

In [9]:
def uhf_price(x):
    return x['volume'] * x['base_clock_price']

In [10]:
# Keep in mind that some stations did not start off-air. However, we do know whether they agreed to possibly being put off-air. 
participation_bounds = stations_df.query('off_air_option == "Y"').copy()
participation_bounds['upper_bound_value'] = participation_bounds['off_air_opening_price']
participation_bounds['lower_bound_value'] = 0
check_unique(participation_bounds)

# NOT ticking the box that you would be OK participating in the auction at off-air was a value statement 
not_participation_bounds = stations_df.query('off_air_option == "N"').copy()
not_participation_bounds['lower_bound_value'] = not_participation_bounds['off_air_opening_price']
not_participation_bounds['upper_bound_value'] = INFINITY
check_unique(not_participation_bounds)


In [11]:
# You didn't participate at all. That must be because the opening price offer for off-air was no good.
price_bounds = prices_df[['facility_id', 'p_open']].copy()
price_bounds = price_bounds[~pd.isnull(price_bounds['p_open'])]
price_bounds = price_bounds[~price_bounds['facility_id'].isin(stations_df['facility_id'].unique())]
price_bounds = price_bounds.rename(columns={'p_open': 'lower_bound_value'})
price_bounds['upper_bound_value'] = INFINITY
check_unique(price_bounds)


In [12]:
# # Any time you acccept an offer for off-air is a valid upper bound, fallback or otherwise
clean_bounds = df[(df['bid_option'] == 'Go off-air') | ((df['bid_option'].str.startswith('Move')) & (df['fallback_option'] == 'Go off-air'))]
clean_bounds = clean_bounds.sort_values('total_round_order').drop_duplicates(subset=['facility_id'], keep='last')
clean_bounds['upper_bound_value'] = clean_bounds[['bid_option_price', 'fallback_option_price']].max(axis=1)

clean_bounds['lower_bound_value'] = 0
check_unique(clean_bounds)

In [13]:
# When a station wants to exit the auction, you know their value is higher than the offered price
# Unfortunately not all prices are listed, but so we don't know what price they faced at the time. But the benchmark overprice is defined as "The per volume difference between the station’s current price and its new price offer for its currently held option". As long as the currently held option remains off-air, benchmark overprice should help us discover the new price!
drop_out_rounds = df[(df['bid_option'] == 'Drop out of bidding') | (df['fallback_option'] == 'Drop out of bidding')].copy()
drop_out_rounds = drop_out_rounds.sort_values('total_round_order').drop_duplicates(subset=['facility_id'], keep='last')
drop_out_rounds['upper_bound_value'] = INFINITY
drop_out_rounds['lower_bound_value'] = drop_out_rounds.apply(uhf_price, axis=1)          
# Zero base clock rounds occassionally don't have benchmark overprice values for some reason... Either way, in a zero base clock round all of your prices are going to be zero. No matter what!
check_unique(drop_out_rounds)

In [14]:
# If you win AND are off-the-air, we learn that your value for your home band is less than what they pay you
w = winners_df.query('winning_bid_option == "Go off-air"')[['facility_id', 'compensation']].copy().rename(columns={'compensation':'upper_bound_value'})
w['lower_bound_value'] = 0
# TODO: Schurz Communications, Inc. seems to have won for $0.... What the hell? Let's just remove that since it will mess everything up
w = w[w['facility_id'] != 35630]
check_unique(w)

In [15]:
tmp = pd.concat((w[COLUMNS], drop_out_rounds[COLUMNS], clean_bounds[COLUMNS], participation_bounds[COLUMNS], not_participation_bounds[COLUMNS], price_bounds[COLUMNS]), sort=True)

# You want the highest lower bound and the lowest upper bound
lbs = tmp.groupby('facility_id')['lower_bound_value'].max()
ubs = tmp.groupby('facility_id')['upper_bound_value'].min()
records = []
for f in tmp['facility_id'].unique():
    records.append({
        'facility_id': f,
        'lower_bound_value': lbs[f],
        'upper_bound_value': ubs[f]
    })
tmp = pd.DataFrame.from_records(records)
check_unique(tmp)

print(tmp['facility_id'].nunique())

1877


In [16]:
data_df = tmp.merge(prices_df[['facility_id', 'population', 'p_open', 'dma']], how='outer')
data_df['participate'] = data_df['facility_id'].apply(lambda x: x in stations_df['facility_id'].unique())
data_df = data_df.merge(prices_df[['facility_id','pre_auction_band']])
data_df = data_df[data_df['pre_auction_band'] == 'UHF']
check_unique(data_df)

In [17]:
data_df.loc[data_df['pre_auction_band'] == 'UHF'].to_csv('UHF_Data_v3.csv', index=False)